In [1]:
#data from landsat id and path row

In [1]:
from datetime import date
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
import folium
import gdal
import time
import requests
import os 
import shutil

In [6]:
download_path='../ege/'
if not os.path.isdir(download_path):
    os.mkdir(download_path)
    

In [3]:
s3_scenes = pd.read_csv('http://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz', compression='gzip')

In [4]:
s3_scenes.download_url

0          https://s3-us-west-2.amazonaws.com/landsat-pds...
1          https://s3-us-west-2.amazonaws.com/landsat-pds...
2          https://s3-us-west-2.amazonaws.com/landsat-pds...
3          https://s3-us-west-2.amazonaws.com/landsat-pds...
4          https://s3-us-west-2.amazonaws.com/landsat-pds...
                                 ...                        
1758237    https://s3-us-west-2.amazonaws.com/landsat-pds...
1758238    https://s3-us-west-2.amazonaws.com/landsat-pds...
1758239    https://s3-us-west-2.amazonaws.com/landsat-pds...
1758240    https://s3-us-west-2.amazonaws.com/landsat-pds...
1758241    https://s3-us-west-2.amazonaws.com/landsat-pds...
Name: download_url, Length: 1758242, dtype: object

In [5]:
target_imgs = []
number_of_images = int(input('How many images do you want to download?:  '))
for i in range(0, number_of_images): 
    img=input('Landsat-8 Image Name (e.g. LC08_L1TP_029029_20190915_20190915_01_RT:  ')
    img.replace(" ","")
    target_imgs.append(img) 


How many images do you want to download?:   2
Landsat-8 Image Name (e.g. LC08_L1TP_029029_20190915_20190915_01_RT:   LC08_L1TP_181033_20190823_20190903_01_T1
Landsat-8 Image Name (e.g. LC08_L1TP_029029_20190915_20190915_01_RT:   LC08_L1TP_181033_20190722_20190801_01_T1


In [8]:
target_imgs

['LC08_L1TP_181033_20190823_20190903_01_T1',
 'LC08_L1TP_181033_20190722_20190801_01_T1']

In [1]:
bilbao=['LC08_L1TP_201030_20190803_20190819_01_T1']

In [7]:
urfa=['LC08_L1TP_173034_20190527_20190605_01_T1',
 'LC08_L1TP_173034_20190628_20190706_01_T1',
 'LC08_L1TP_173034_20190831_20190916_01_T1']

In [9]:
ege=['LC08_L1TP_181033_20190823_20190903_01_T1',
 'LC08_L1TP_181033_20190722_20190801_01_T1']



In [7]:
len(target_imgs)

3

In [10]:
def DefineImages(target_imgs):
    bulk_list=[]
    if len(target_imgs)==1:
        scenes=s3_scenes[s3_scenes['productId']==target_imgs[0]]
        bulk_list.append(scenes.iloc[0])
        bulk_frame = pd.concat(bulk_list, 1).T
    else:
        for i in target_imgs:
            scenes=s3_scenes[s3_scenes['productId']==i]
            bulk_list.append(scenes.iloc[0])
        
        bulk_frame = pd.concat(bulk_list, 1).T
    return bulk_frame

In [11]:
bulk_frame=DefineImages(target_imgs)


In [12]:
bulk_frame

,productId,entityId,acquisitionDate,cloudCover,processingLevel,path,row,min_lat,min_lon,max_lat,max_lon,download_url
1687893,LC08_L1TP_181033_20190823_20190903_01_T1,LC81810332019235LGN00,2019-08-23 08:52:32.626804,0.35,L1TP,181,33,37.8252,25.3049,39.9519,27.9932,https://s3-us-west-2.amazonaws.com/landsat-pds...
1652695,LC08_L1TP_181033_20190722_20190801_01_T1,LC81810332019203LGN00,2019-07-22 08:52:21.298923,0.05,L1TP,181,33,37.8253,25.3084,39.952,27.9966,https://s3-us-west-2.amazonaws.com/landsat-pds...


In [13]:
def ImageDownload(bandlist,bulk_frame,download_path='./'):
    start_time = time.time()
    for i,row in bulk_frame.iterrows():
                
        for band in bandlist:                                                                  
            response = requests.get(row.download_url)
            if response.status_code == 200:
                if os.path.exists(download_path+row.productId)==False:
                    os.makedirs(download_path+row.productId)
                    
                
                image_dir=download_path+row.productId+'/'
                print(image_dir)
                url = row.download_url[:-10]
                image = row.productId+'_B{}.TIF'.format(band)
                image_path=image_dir+image
                response = requests.get(url+image, stream=True)
                #translate_path=image_dir+'4326/'+image
                #print(translate_path)
                
                with open(os.path.join(image_path), 'wb') as output:
                    shutil.copyfileobj(response.raw, output)
                    #gdal.Warp(translate_path ,image_path, format = 'GTiff', dstSRS = 'EPSG:4326',srcNodata = 0,dstNodata=0)
                
                
                del response
                
        metadata=row.productId+'_MTL.json'
        response_MTL = requests.get(url+metadata, stream=True)
    
        with open(os.path.join(image_dir+metadata), 'wb') as output:
            shutil.copyfileobj(response_MTL.raw, output)
        del response_MTL
        print(image_dir+metadata)
        quality_ass=row.productId+'_BQA.TIF'
        response_QA = requests.get(url+quality_ass, stream=True)
        
        with open(os.path.join(image_dir+quality_ass), 'wb') as output:
            shutil.copyfileobj(response_QA.raw, output)
        print(image_dir+quality_ass)
    print("--- %s seconds ---" % (time.time() - start_time))
        

In [14]:
bandlist=[2,3,4,5,6,7,8,9,10]

In [15]:
ImageDownload(bandlist,bulk_frame,download_path)

../tez_data/ege/LC08_L1TP_181033_20190823_20190903_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190823_20190903_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190823_20190903_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190823_20190903_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190823_20190903_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190823_20190903_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190823_20190903_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190823_20190903_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190823_20190903_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190823_20190903_01_T1/LC08_L1TP_181033_20190823_20190903_01_T1_MTL.json
../tez_data/ege/LC08_L1TP_181033_20190823_20190903_01_T1/LC08_L1TP_181033_20190823_20190903_01_T1_BQA.TIF
../tez_data/ege/LC08_L1TP_181033_20190722_20190801_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190722_20190801_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190722_20190801_01_T1/
../tez_data/ege/LC08_L1TP_181033_20190722_20190801_01_T1/
../tez_data/ege/LC08_L1TP_181033_

In [28]:
import rasterio

In [31]:
def ImageInfo(imagepath):
    raster=rasterio.open(imagepath)
    rasterinfo={'profile':raster.profile,'name':raster.name,'mode':raster.mode,'band_count':raster.count,
               'size':(raster.width,raster.height),'bbox':raster.bounds,'crs':raster.crs}
    return rasterinfo